In [1]:
%cd ..

/Users/treycole/Codes/WanPy


In [2]:
from WanPy.pythtb_Wannier import *
import WanPy.models as models
import WanPy.plotting as plot
from itertools import product

from pythtb import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
import scipy

In [3]:
def Haldane(delta, t, t2, a=1):
    lat = a * np.array([[3/2, np.sqrt(3)/2], [3/2, -np.sqrt(3)/2]])
    # lat = [[1, 0],[0.5, np.sqrt(3)/2]]
    orb = [[1/3, 1/3],[2/3, 2/3]]

    model = tb_model(2, 2, lat, orb)

    model.set_onsite([-delta, delta], mode='reset')

    for lvec in ([0, 0], [-1, 0], [0, -1]):
        model.set_hop(t, 0, 1, lvec, mode='reset')
        model.set_hop(t, 0, 1, lvec, mode='reset')

    for lvec in ([1, 0], [-1, 1], [0, -1]):
        model.set_hop(t2*1j, 0, 0, lvec, mode='reset')
        model.set_hop(t2*-1j, 1, 1, lvec, mode='reset')

    return model

# latvecs = np.array([[3/2, np.sqrt(3)/2], [3/2, -np.sqrt(3)/2]])
# recipvecs = (2*np.pi/3) * np.array([[1, np.sqrt(3)], [1, -np.sqrt(3)]])

In [39]:
def get_recip_lat_vecs(lat_vecs):
    b = 2 * np.pi * np.linalg.inv(lat_vecs).T
    return b

def get_k_shell(*nks, model, N_sh, tol_dp=8, report=False):

    lat_vecs = model.get_lat() # lattice vectors
    recip_vecs = get_recip_lat_vecs(lat_vecs)
    dk = np.array([recip_vecs[i]/nk for i, nk in enumerate(nks)])
    
    # vectors of integers multiplying dk 
    nnbr_idx = list(product(list(range(-N_sh, N_sh+1)), repeat=len(nks)))
    nnbr_idx.remove((0,0))
    nnbr_idx = np.array(nnbr_idx)

    # vectors connecting k-points near Gamma point
    b_vecs = np.array([nnbr_idx[i] @ dk for i in range(nnbr_idx.shape[0])]) 
    dists = np.array([np.vdot(b_vecs[i], b_vecs[i]) for i in range(b_vecs.shape[0])])
    dists = dists.round(tol_dp) 

    # sorting by distance
    sorted_idxs = np.argsort(dists)
    dists_sorted = dists[sorted_idxs]
    b_vecs_sorted = b_vecs[sorted_idxs]
    nnbr_idx_sorted = nnbr_idx[sorted_idxs]

    # keep only b_vecs in N_sh shells 
    unique_dists = sorted(list(set(dists)))
    keep_dists = unique_dists[:N_sh]
    k_shell = [b_vecs_sorted[np.isin(dists_sorted, keep_dists[i])] for i in range(len(keep_dists))]
    idx_shell = [nnbr_idx_sorted[np.isin(dists_sorted, keep_dists[i])] for i in range(len(keep_dists))]

    if report:
        dist_degen = {ud: len(k_shell[i]) for i, ud in enumerate(keep_dists)}
        print(f"Shell distances and degeneracies: {dist_degen}")
        print(f"k-shells: {k_shell}")
        print(f"idx-shells: {idx_shell}")
        
    return k_shell, idx_shell
        

def get_weights(*nks, model, N_sh=1, report=False):
    k_shell, _ = get_k_shell(*nks, model=model, N_sh=N_sh, report=report)
    dim_k = len(nks)
    Cart_idx = list(comb(range(dim_k), 2))
    n_comb = len(Cart_idx)

    A = np.zeros((n_comb, N_sh))
    q = np.zeros((n_comb))

    for j, (alpha, beta) in enumerate(Cart_idx):
        if alpha == beta:
            q[j] = 1
        for s in range(N_sh):
            b_star = k_shell[s]
            for i in range(b_star.shape[0]):
                b = b_star[i]
                A[j, s] += b[alpha] * b[beta]
                
    U, D, Vt = np.linalg.svd(A, full_matrices=False)
    w = (Vt.T @ np.linalg.inv(np.diag(D)) @ U.T) @ q

    return w

In [77]:
# model = models.chessboard(0, 0, 0)#.make_supercell([[2,0], [0,2]])
model = models.Haldane(0, 0, 0)#.make_supercell([[2,0], [0,2]])
lat_vecs = model.get_lat() # lattice vectors
recip_vecs = get_recip_lat_vecs(lat_vecs)
orbs = model.get_orb()

recip_vecs

array([[ 6.28318531, -3.62759873],
       [ 0.        ,  7.25519746]])

In [83]:
N_sh = 1
nks = 6, 6
tol_dp = 8

dk = np.array([recip_vecs[i]/nk for i, nk in enumerate(nks)])
nnbr_mask = list(product(list(range(-N_sh, N_sh+1)), repeat=len(nks)))
nnbr_mask.remove((0,0))
nnbr_mask = np.array(nnbr_mask)
bs = np.array([nnbr_mask[i] @ dk for i in range(nnbr_mask.shape[0])]) 
dists = np.array([np.vdot(bs[i], bs[i]) for i in range(bs.shape[0])])
dists = dists.round(tol_dp)
sorted_idxs = np.argsort(dists)
dists_sorted = dists[sorted_idxs]
bs_sorted = bs[sorted_idxs]
mask_sorted = nnbr_mask[sorted_idxs]

unique_dists = sorted(list(set(dists)))
num_shells = len(unique_dists)
keep_dists = unique_dists[:N_sh]
k_shell = [bs_sorted[np.isin(dists_sorted, keep_dists[i])] for i in range(len(keep_dists))]
idx_shell = [mask_sorted[np.isin(dists_sorted, keep_dists[i])] for i in range(len(keep_dists))]
dist_degen = {ud: len(k_shell[i]) for i, ud in enumerate(keep_dists)}

print(f"Distances and degeneracies: {dist_degen}")
print(f"k-shells: {k_shell}")
print(f"idx-shells: {idx_shell}")

Distances and degeneracies: {1.46216361: 6}
k-shells: [array([[-1.04719755, -0.60459979],
       [-1.04719755,  0.60459979],
       [ 0.        , -1.20919958],
       [ 0.        ,  1.20919958],
       [ 1.04719755, -0.60459979],
       [ 1.04719755,  0.60459979]])]
idx-shells: [array([[-1, -1],
       [-1,  0],
       [ 0, -1],
       [ 0,  1],
       [ 1,  0],
       [ 1,  1]])]


In [84]:
k_shell[0]

array([[-1.04719755, -0.60459979],
       [-1.04719755,  0.60459979],
       [ 0.        , -1.20919958],
       [ 0.        ,  1.20919958],
       [ 1.04719755, -0.60459979],
       [ 1.04719755,  0.60459979]])

In [85]:
w_b = get_weights(*nks, model=model, N_sh=N_sh, report=True)
w_b

Shell distances and degeneracies: {1.46216361: 6}
k-shells: [array([[-1.04719755, -0.60459979],
       [-1.04719755,  0.60459979],
       [ 0.        , -1.20919958],
       [ 0.        ,  1.20919958],
       [ 1.04719755, -0.60459979],
       [ 1.04719755,  0.60459979]])]
idx-shells: [array([[-1, -1],
       [-1,  0],
       [ 0, -1],
       [ 0,  1],
       [ 1,  0],
       [ 1,  1]])]


array([0.22797266])

In [86]:
1/(2*(np.vdot(bs[0,0], bs[0,0])))

0.45594532639052004

In [87]:
res = 0
alpha = 1
beta = 1

for s in range(len(k_shell)):
    b_star = k_shell[s]
    # print(b_star)
    for i in range(b_star.shape[0]):
        b = b_star[i]
        res += w_b[s] * b[alpha] * b[beta]

res

1.0

In [88]:
recip_vecs

array([[ 6.28318531, -3.62759873],
       [ 0.        ,  7.25519746]])

In [89]:
orbs

array([[0.33333333, 0.33333333],
       [0.66666667, 0.66666667]])

In [100]:
x = 4
(x-x%2)/2

2.0

In [101]:
x //2

2

In [90]:
k_idx_arr = list(product(*[range(nk) for nk in nks])) # list of all k_indices
for k_idx in k_idx_arr:
    k_pt = k_idx @ dk
    for idx, idx_vec in enumerate(idx_shell[0]): # nearest neighbors
        k_nbr_idx = np.array(k_idx) + idx_vec
        k_nbr = k_nbr_idx @ dk
        # print(idx_vec, k_nbr)
        # if the translated k-index contains the -1st or last_idx+1 then we crossed the BZ boundary
        # and need to include phases
        cross_bndry = True if np.any(np.in1d(k_nbr_idx, [-1, *nks])) else False
        if cross_bndry:
            mod_idx = np.mod(k_nbr_idx, nks)
            diff = k_nbr_idx - mod_idx
            G = np.divide(np.array(diff), np.array(nks))
            print("idx_vec: ", idx_vec)
            print(f"k_idx: {k_idx}, k_nbr_idx: {k_nbr_idx}")
            print("k_nbr point: ", k_nbr)
            print("mod", mod_idx)
            print("diff", diff)
            print("G",  G)
            print(orbs @ G.T)
            print()

idx_vec:  [-1 -1]
k_idx: (0, 0), k_nbr_idx: [-1 -1]
k_nbr point:  [-1.04719755 -0.60459979]
mod [5 5]
diff [-6 -6]
G [-1. -1.]
[-0.66666667 -1.33333333]

idx_vec:  [-1  0]
k_idx: (0, 0), k_nbr_idx: [-1  0]
k_nbr point:  [-1.04719755  0.60459979]
mod [5 0]
diff [-6  0]
G [-1.  0.]
[-0.33333333 -0.66666667]

idx_vec:  [ 0 -1]
k_idx: (0, 0), k_nbr_idx: [ 0 -1]
k_nbr point:  [ 0.         -1.20919958]
mod [0 5]
diff [ 0 -6]
G [ 0. -1.]
[-0.33333333 -0.66666667]

idx_vec:  [-1 -1]
k_idx: (0, 1), k_nbr_idx: [-1  0]
k_nbr point:  [-1.04719755  0.60459979]
mod [5 0]
diff [-6  0]
G [-1.  0.]
[-0.33333333 -0.66666667]

idx_vec:  [-1  0]
k_idx: (0, 1), k_nbr_idx: [-1  1]
k_nbr point:  [-1.04719755  1.81379936]
mod [5 1]
diff [-6  0]
G [-1.  0.]
[-0.33333333 -0.66666667]

idx_vec:  [-1 -1]
k_idx: (0, 2), k_nbr_idx: [-1  1]
k_nbr point:  [-1.04719755  1.81379936]
mod [5 1]
diff [-6  0]
G [-1.  0.]
[-0.33333333 -0.66666667]

idx_vec:  [-1  0]
k_idx: (0, 2), k_nbr_idx: [-1  2]
k_nbr point:  [-1.047197

In [206]:
nks = 10, 10, 10
# define lattice vectors
lat=[[1.0, 0.0, 0], [0.0, 1.0, 0], [0, 0, 1.0]]
# define coordinates of orbitals

# make two dimensional tight-binding checkerboard model
model = tb_model(3, 3, lat=lat)

# orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors
b_vec = get_recip_lat_vecs(model)

 Orbital positions not specified. I will assume a single orbital at the origin.


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [52]:
lat_vecs[0]

array([1., 0., 0.])

In [53]:
b_vec[0]

array([6.28318531, 0.        , 0.        ])

In [54]:
b_vec[1] @ lat_vecs[1] - 2*np.pi

0.0

In [55]:
dk = np.array([b_vec[i]/nk for i, nk in enumerate(nks)])
dk

array([[0.62831853, 0.        , 0.        ],
       [0.        , 0.62831853, 0.        ],
       [0.        , 0.        , 0.62831853]])

In [58]:
w = get_weights(*nks, model=model)
w/2

(10, 10, 10) [[6.28318531 0.         0.        ]
 [0.         6.28318531 0.        ]
 [0.         0.         6.28318531]]
[[0.62831853 0.         0.        ]
 [0.         0.62831853 0.        ]
 [0.         0.         0.62831853]]
[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]


array([1.2665148])

In [57]:
# 3D k-mesh
Z = 6 # number of nearest neighbors in reciprocal lattice
G = 2*np.pi/lat_vecs[0][0] # reciprocal lattice vector
b = G / nks[0]  # spacing between allowed kx
w_b = 3/(Z*b**2) # finite difference weights for cubic lattice
w_b

1.2665147955292224